In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/home/mothikaa/ANPR/images/N192.xml,543,769,390,461
1,/home/mothikaa/ANPR/images/N149.xml,186,324,116,148
2,/home/mothikaa/ANPR/images/N100.xml,134,301,312,350
3,/home/mothikaa/ANPR/images/N116.xml,157,316,226,278
4,/home/mothikaa/ANPR/images/N57.xml,211,406,161,202


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'/home/mothikaa/ANPR/images/N192.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/home/mothikaa/ANPR/images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'/home/mothikaa/ANPR/images/N192.jpeg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['/home/mothikaa/ANPR/images/N192.jpeg',
 '/home/mothikaa/ANPR/images/N149.jpeg',
 '/home/mothikaa/ANPR/images/N100.jpeg',
 '/home/mothikaa/ANPR/images/N116.jpeg',
 '/home/mothikaa/ANPR/images/N57.jpeg',
 '/home/mothikaa/ANPR/images/N206.jpeg',
 '/home/mothikaa/ANPR/images/N199.jpeg',
 '/home/mothikaa/ANPR/images/N123.jpeg',
 '/home/mothikaa/ANPR/images/N78.jpeg',
 '/home/mothikaa/ANPR/images/N66.jpeg',
 '/home/mothikaa/ANPR/images/N90.jpeg',
 '/home/mothikaa/ANPR/images/N148.jpeg',
 '/home/mothikaa/ANPR/images/N150.jpeg',
 '/home/mothikaa/ANPR/images/N126.jpeg',
 '/home/mothikaa/ANPR/images/N173.jpeg',
 '/home/mothikaa/ANPR/images/N9.jpeg',
 '/home/mothikaa/ANPR/images/N5.jpeg',
 '/home/mothikaa/ANPR/images/N68.jpeg',
 '/home/mothikaa/ANPR/images/N4.jpeg',
 '/home/mothikaa/ANPR/images/N225.jpeg',
 '/home/mothikaa/ANPR/images/N140.jpeg',
 '/home/mothikaa/ANPR/images/N122.jpeg',
 '/home/mothikaa/ANPR/images/N132.jpeg',
 '/home/mothikaa/ANPR/images/N232.jpeg',
 '/home/mothikaa/ANPR/image

## Verify Image and Output

In [8]:
file_path = image_path[0]
file_path

'/home/mothikaa/ANPR/images/N192.jpeg'

In [9]:
img = cv2.imread(file_path)
cv2.rectangle(img,(543,390),(769,461),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_AUTOSIZE)
cv2.startWindowThread()
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Data preprocessing

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:
labels = df.iloc[:,1:].values

In [12]:
data = []
output = []

for ind in range(len(image_path)):
    image = image_path[0]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape

    # Preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr

    #Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax)
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [13]:
output

[(0.4344, 0.6152, 0.5547652916073968, 0.6557610241820768),
 (0.1488, 0.2592, 0.16500711237553342, 0.21052631578947367),
 (0.1072, 0.2408, 0.4438122332859175, 0.49786628733997157),
 (0.1256, 0.2528, 0.32147937411095306, 0.39544807965860596),
 (0.1688, 0.3248, 0.22901849217638692, 0.28733997155049784),
 (0.0968, 0.172, 0.17069701280227595, 0.20910384068278806),
 (0.036, 0.0896, 0.2930298719772404, 0.3627311522048364),
 (0.0088, 0.1968, 0.11095305832147938, 0.23470839260312945),
 (0.1168, 0.2416, 0.22759601706970128, 0.27880512091038406),
 (0.1304, 0.256, 0.35135135135135137, 0.39829302987197723),
 (0.0424, 0.148, 0.2930298719772404, 0.33001422475106684),
 (0.1952, 0.2952, 0.3413940256045519, 0.4167852062588905),
 (0.1376, 0.2976, 0.1379800853485064, 0.2034139402560455),
 (0.1192, 0.3192, 0.38264580369843526, 0.465149359886202),
 (0.064, 0.268, 0.21337126600284495, 0.3456614509246088),
 (0.1136, 0.3528, 0.07823613086770982, 0.2375533428165007),
 (0.7504, 0.9384, 1.0085348506401137, 1.0839

In [14]:
x = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [15]:
x.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# Deep Learning Model

In [17]:
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [18]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable = False
#----------------------------------------------------------------------------------------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
#-----------------------------------------Model
model = Model(inception_resnet.input,outputs=headmodel)

In [19]:
# Compile Model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

activation_181 (Activation)     (None, 5, 5, 224)    0           batch_normalization_181[0][0]    
__________________________________________________________________________________________________
conv2d_179 (Conv2D)             (None, 5, 5, 192)    399360      block8_4_ac[0][0]                
__________________________________________________________________________________________________
conv2d_182 (Conv2D)             (None, 5, 5, 256)    172032      activation_181[0][0]             
__________________________________________________________________________________________________
batch_normalization_179 (BatchN (None, 5, 5, 192)    576         conv2d_179[0][0]                 
__________________________________________________________________________________________________
batch_normalization_182 (BatchN (None, 5, 5, 256)    768         conv2d_182[0][0]                 
__________________________________________________________________________________________________
activation

# Model Training

In [20]:
from tensorflow.keras.callbacks import TensorBoard

In [21]:
tfb = TensorBoard('object_detection')

In [22]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                   validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 [==============================] - 39s 2s/step - loss: 0.3364 - val_loss: 0.4146
Epoch 2/100
18/18 [==============================] - 25s 1s/step - loss: 0.3717 - val_loss: 0.4146
Epoch 3/100
18/18 [==============================] - 24s 1s/step - loss: 0.3551 - val_loss: 0.4146
Epoch 4/100
18/18 [==============================] - 23s 1s/step - loss: 0.3257 - val_loss: 0.4146
Epoch 5/100
18/18 [==============================] - 24s 1s/step - loss: 0.3260 - val_loss: 0.4146
Epoch 6/100
18/18 [==============================] - 24s 1s/step - loss: 0.3344 - val_loss: 0.4146
Epoch 7/100
18/18 [==============================] - 24s 1s/step - loss: 0.3326 - val_loss: 0.4146
Epoch 8/100
18/18 [==============================] - 25s 1s/step - loss: 0.3339 - val_loss: 0.4146
Epoch 9/100
18/18 [==============================] - 24s 1s/step - loss: 0.3260 - val_loss: 0.4146
Epoch 10/100
18/18 [==============================] - 25s 1s/step - loss: 0.3306 - val_loss: 0.4146
Epoch 11/

18/18 [==============================] - 21s 1s/step - loss: 0.3572 - val_loss: 0.4146
Epoch 84/100
18/18 [==============================] - 21s 1s/step - loss: 0.3390 - val_loss: 0.4146
Epoch 85/100
18/18 [==============================] - 21s 1s/step - loss: 0.3447 - val_loss: 0.4146
Epoch 86/100
18/18 [==============================] - 22s 1s/step - loss: 0.3201 - val_loss: 0.4146
Epoch 87/100
18/18 [==============================] - 22s 1s/step - loss: 0.3358 - val_loss: 0.4146
Epoch 88/100
18/18 [==============================] - 22s 1s/step - loss: 0.3348 - val_loss: 0.4146
Epoch 89/100
18/18 [==============================] - 22s 1s/step - loss: 0.3249 - val_loss: 0.4146
Epoch 90/100
18/18 [==============================] - 22s 1s/step - loss: 0.3590 - val_loss: 0.4146
Epoch 91/100
18/18 [==============================] - 22s 1s/step - loss: 0.3582 - val_loss: 0.4146
Epoch 92/100
18/18 [==============================] - 22s 1s/step - loss: 0.3208 - val_loss: 0.4146
Epoch 93/100


In [23]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                   validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
18/18 [==============================] - 23s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 103/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 104/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 105/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 106/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 107/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 108/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 109/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 110/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 111/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_los

18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 184/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 185/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 186/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 187/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 188/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 189/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 190/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 191/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoch 192/200
18/18 [==============================] - 22s 1s/step - loss: 0.3321 - val_loss: 0.4146
Epoc

In [24]:
model.save('/home/mothikaa/ANPR/models/object_detection.h5')